# Chapter 10: Inheritance and Polymorphism

## Introduction to Object-Oriented Programming in SystemVerilog

SystemVerilog supports object-oriented programming (OOP) concepts that enable code reusability, modularity, and better organization. This chapter explores inheritance and polymorphism, two fundamental OOP concepts that allow you to create hierarchical relationships between classes and write more flexible, maintainable code.

## Class Inheritance (extends)

Inheritance allows you to create new classes based on existing classes, inheriting their properties and methods while adding new functionality or modifying existing behavior.

### Basic Inheritance Syntax

```systemverilog
class BaseClass;
    // Base class properties and methods
endclass

class DerivedClass extends BaseClass;
    // Derived class inherits from BaseClass
    // Additional properties and methods
endclass
```

### Practical Example: Vehicle Hierarchy

```systemverilog
// Base Vehicle class
class Vehicle;
    string make;
    string model;
    int year;
    
    function new(string mk = "Unknown", string md = "Unknown", int yr = 2024);
        make = mk;
        model = md;
        year = yr;
    endfunction
    
    virtual function void display_info();
        $display("Vehicle: %s %s (%0d)", make, model, year);
    endfunction
    
    virtual function void start_engine();
        $display("Starting engine...");
    endfunction
endclass

// Car class inherits from Vehicle
class Car extends Vehicle;
    int num_doors;
    string fuel_type;
    
    function new(string mk = "Unknown", string md = "Unknown", 
                 int yr = 2024, int doors = 4, string fuel = "Gasoline");
        super.new(mk, md, yr);  // Call parent constructor
        num_doors = doors;
        fuel_type = fuel;
    endfunction
    
    // Override parent method
    virtual function void display_info();
        super.display_info();  // Call parent method
        $display("  Type: Car, Doors: %0d, Fuel: %s", num_doors, fuel_type);
    endfunction
    
    function void open_trunk();
        $display("Opening car trunk...");
    endfunction
endclass

// Motorcycle class inherits from Vehicle
class Motorcycle extends Vehicle;
    bit has_sidecar;
    int engine_cc;
    
    function new(string mk = "Unknown", string md = "Unknown", 
                 int yr = 2024, bit sidecar = 0, int cc = 250);
        super.new(mk, md, yr);
        has_sidecar = sidecar;
        engine_cc = cc;
    endfunction
    
    virtual function void display_info();
        super.display_info();
        $display("  Type: Motorcycle, Engine: %0dcc, Sidecar: %s", 
                 engine_cc, has_sidecar ? "Yes" : "No");
    endfunction
    
    virtual function void start_engine();
        $display("Kick-starting motorcycle engine...");
    endfunction
endclass
```

## Method Overriding

Method overriding allows derived classes to provide specific implementations of methods defined in their parent classes.

### Rules for Method Overriding

1. The method signature must match exactly
2. Use the `virtual` keyword in the base class
3. The derived class method automatically becomes virtual

### Example: Shape Hierarchy with Method Overriding

```systemverilog
// Base Shape class
class Shape;
    string name;
    
    function new(string n = "Shape");
        name = n;
    endfunction
    
    // Virtual method to be overridden
    virtual function real calculate_area();
        $display("Warning: calculate_area() not implemented for %s", name);
        return 0.0;
    endfunction
    
    virtual function void draw();
        $display("Drawing a generic %s", name);
    endfunction
endclass

// Rectangle class
class Rectangle extends Shape;
    real width, height;
    
    function new(real w = 1.0, real h = 1.0);
        super.new("Rectangle");
        width = w;
        height = h;
    endfunction
    
    // Override calculate_area method
    virtual function real calculate_area();
        return width * height;
    endfunction
    
    virtual function void draw();
        $display("Drawing rectangle: %0.2f x %0.2f", width, height);
    endfunction
endclass

// Circle class
class Circle extends Shape;
    real radius;
    
    function new(real r = 1.0);
        super.new("Circle");
        radius = r;
    endfunction
    
    virtual function real calculate_area();
        return 3.14159 * radius * radius;
    endfunction
    
    virtual function void draw();
        $display("Drawing circle with radius: %0.2f", radius);
    endfunction
endclass

// Triangle class
class Triangle extends Shape;
    real base, height;
    
    function new(real b = 1.0, real h = 1.0);
        super.new("Triangle");
        base = b;
        height = h;
    endfunction
    
    virtual function real calculate_area();
        return 0.5 * base * height;
    endfunction
    
    virtual function void draw();
        $display("Drawing triangle: base=%0.2f, height=%0.2f", base, height);
    endfunction
endclass
```

## The super Keyword

The `super` keyword provides access to the parent class's methods and properties from within a derived class.

### Uses of super

1. **Calling parent constructor**: `super.new()`
2. **Calling parent methods**: `super.method_name()`
3. **Accessing parent properties**: `super.property_name`

### Example: Employee Hierarchy

```systemverilog
class Employee;
    string name;
    int employee_id;
    real base_salary;
    
    function new(string n, int id, real salary);
        name = n;
        employee_id = id;
        base_salary = salary;
    endfunction
    
    virtual function real calculate_pay();
        return base_salary;
    endfunction
    
    virtual function void display_info();
        $display("Employee: %s (ID: %0d), Base Salary: $%0.2f", 
                 name, employee_id, base_salary);
    endfunction
endclass

class Manager extends Employee;
    real bonus_percentage;
    int team_size;
    
    function new(string n, int id, real salary, real bonus = 0.15, int team = 5);
        super.new(n, id, salary);  // Call parent constructor
        bonus_percentage = bonus;
        team_size = team;
    endfunction
    
    virtual function real calculate_pay();
        real base_pay = super.calculate_pay();  // Get base salary from parent
        return base_pay + (base_pay * bonus_percentage);
    endfunction
    
    virtual function void display_info();
        super.display_info();  // Call parent display method
        $display("  Role: Manager, Team Size: %0d, Bonus: %0.1f%%", 
                 team_size, bonus_percentage * 100);
    endfunction
    
    function void conduct_meeting();
        $display("%s is conducting a team meeting", name);
    endfunction
endclass

class Developer extends Employee;
    string programming_language;
    int projects_completed;
    
    function new(string n, int id, real salary, string lang = "SystemVerilog");
        super.new(n, id, salary);
        programming_language = lang;
        projects_completed = 0;
    endfunction
    
    virtual function real calculate_pay();
        real base_pay = super.calculate_pay();
        real project_bonus = projects_completed * 500.0;  // $500 per project
        return base_pay + project_bonus;
    endfunction
    
    virtual function void display_info();
        super.display_info();
        $display("  Role: Developer, Language: %s, Projects: %0d", 
                 programming_language, projects_completed);
    endfunction
    
    function void complete_project();
        projects_completed++;
        $display("%s completed a project in %s", name, programming_language);
    endfunction
endclass
```

## Virtual Methods

Virtual methods enable polymorphism by allowing method calls to be resolved at runtime based on the actual object type.

### Virtual Method Rules

1. Use `virtual` keyword in the base class method declaration
2. Derived class methods that override virtual methods are automatically virtual
3. Virtual methods enable dynamic binding

### Example: Communication Protocol Stack

```systemverilog
// Base Protocol class
class Protocol;
    string protocol_name;
    int header_size;
    
    function new(string name = "Generic", int hdr_size = 0);
        protocol_name = name;
        header_size = hdr_size;
    endfunction
    
    // Virtual methods for protocol operations
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("Generic encoding for %s protocol", protocol_name);
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("Generic decoding for %s protocol", protocol_name);
    endfunction
    
    virtual function int get_overhead();
        return header_size;
    endfunction
    
    virtual function void display_info();
        $display("Protocol: %s, Header Size: %0d bytes", protocol_name, header_size);
    endfunction
endclass

// TCP Protocol
class TCP_Protocol extends Protocol;
    int sequence_number;
    int window_size;
    
    function new();
        super.new("TCP", 20);  // TCP header is 20 bytes minimum
        sequence_number = 0;
        window_size = 65535;
    endfunction
    
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("TCP: Adding sequence number %0d and checksum", sequence_number);
        sequence_number++;
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("TCP: Verifying checksum and sequence number");
    endfunction
    
    virtual function int get_overhead();
        return super.get_overhead() + 4;  // Additional TCP options
    endfunction
endclass

// UDP Protocol
class UDP_Protocol extends Protocol;
    function new();
        super.new("UDP", 8);  // UDP header is 8 bytes
    endfunction
    
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("UDP: Adding simple header with length and checksum");
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("UDP: Basic header validation");
    endfunction
endclass

// HTTP Protocol (application layer)
class HTTP_Protocol extends Protocol;
    string method;
    string url;
    
    function new(string http_method = "GET", string request_url = "/");
        super.new("HTTP", 0);  // Variable header size
        method = http_method;
        url = request_url;
    endfunction
    
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("HTTP: Creating %s request for %s", method, url);
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("HTTP: Parsing request/response headers");
    endfunction
    
    virtual function int get_overhead();
        return method.len() + url.len() + 20;  // Estimated header size
    endfunction
endclass
```

## Abstract Classes

While SystemVerilog doesn't have explicit abstract class syntax, you can create abstract-like behavior using pure virtual methods and base classes that shouldn't be instantiated directly.

### Abstract Class Pattern

```systemverilog
// Abstract Database Connection class
class DatabaseConnection;
    string connection_string;
    bit connected;
    
    function new(string conn_str);
        connection_string = conn_str;
        connected = 0;
    endfunction
    
    // Pure virtual methods (must be implemented by derived classes)
    pure virtual function bit connect();
    pure virtual function void disconnect();
    pure virtual function string execute_query(string query);
    pure virtual function bit is_connected();
    
    // Concrete method that can be inherited
    function void log_operation(string operation);
        $display("[%0t] Database Operation: %s", $time, operation);
    endfunction
endclass

// MySQL Database implementation
class MySQLConnection extends DatabaseConnection;
    int port;
    string database_name;
    
    function new(string host, int p = 3306, string db = "test");
        super.new($sformatf("mysql://%s:%0d/%s", host, p, db));
        port = p;
        database_name = db;
    endfunction
    
    // Implement abstract methods
    virtual function bit connect();
        log_operation("Connecting to MySQL");
        connected = 1;
        $display("Connected to MySQL database: %s", database_name);
        return connected;
    endfunction
    
    virtual function void disconnect();
        if (connected) begin
            log_operation("Disconnecting from MySQL");
            connected = 0;
            $display("Disconnected from MySQL");
        end
    endfunction
    
    virtual function string execute_query(string query);
        if (!connected) begin
            $display("Error: Not connected to database");
            return "";
        end
        log_operation($sformatf("Executing: %s", query));
        return "MySQL query result";
    endfunction
    
    virtual function bit is_connected();
        return connected;
    endfunction
endclass

// PostgreSQL Database implementation
class PostgreSQLConnection extends DatabaseConnection;
    string schema_name;
    
    function new(string host, string schema = "public");
        super.new($sformatf("postgresql://%s/%s", host, schema));
        schema_name = schema;
    endfunction
    
    virtual function bit connect();
        log_operation("Connecting to PostgreSQL");
        connected = 1;
        $display("Connected to PostgreSQL schema: %s", schema_name);
        return connected;
    endfunction
    
    virtual function void disconnect();
        if (connected) begin
            log_operation("Disconnecting from PostgreSQL");
            connected = 0;
            $display("Disconnected from PostgreSQL");
        end
    endfunction
    
    virtual function string execute_query(string query);
        if (!connected) begin
            $display("Error: Not connected to database");
            return "";
        end
        log_operation($sformatf("Executing on schema %s: %s", schema_name, query));
        return "PostgreSQL query result";
    endfunction
    
    virtual function bit is_connected();
        return connected;
    endfunction
endclass
```

## Polymorphism Examples

Polymorphism allows objects of different types to be treated uniformly through a common interface, with method calls resolved at runtime based on the actual object type.

### Example 1: Graphics Rendering System

```systemverilog
// Test module demonstrating polymorphism with shapes
module polymorphism_demo;
    
    // Array of shape handles (polymorphic collection)
    Shape shapes[];
    Rectangle rect;
    Circle circ;
    Triangle tri;
    
    initial begin
        // Create different shape objects
        rect = new(5.0, 3.0);
        circ = new(2.5);
        tri = new(4.0, 6.0);
        
        // Store them in polymorphic array
        shapes = new[3];
        shapes[0] = rect;  // Rectangle assigned to Shape handle
        shapes[1] = circ;  // Circle assigned to Shape handle
        shapes[2] = tri;   // Triangle assigned to Shape handle
        
        $display("=== Polymorphic Shape Processing ===");
        
        // Process all shapes polymorphically
        foreach (shapes[i]) begin
            $display("\nShape %0d:", i+1);
            shapes[i].draw();  // Calls appropriate draw method
            $display("Area: %0.2f", shapes[i].calculate_area());  // Calls appropriate calculate_area
        end
        
        // Calculate total area
        real total_area = 0.0;
        foreach (shapes[i]) begin
            total_area += shapes[i].calculate_area();
        end
        $display("\nTotal area of all shapes: %0.2f", total_area);
    end
endmodule
```

### Example 2: Network Protocol Handler

```systemverilog
// Protocol handler demonstrating polymorphism
class NetworkStack;
    Protocol protocols[];
    
    function new();
        protocols = new[3];
        protocols[0] = new TCP_Protocol();
        protocols[1] = new UDP_Protocol();
        protocols[2] = new HTTP_Protocol("POST", "/api/data");
    endfunction
    
    // Process packet through all protocol layers
    function void process_packet(ref bit [7:0] data[]);
        $display("=== Processing Packet Through Network Stack ===");
        
        foreach (protocols[i]) begin
            $display("\n--- Layer %0d ---", i+1);
            protocols[i].display_info();
            protocols[i].encode_packet(data);
            $display("Overhead: %0d bytes", protocols[i].get_overhead());
        end
        
        $display("\n=== Decoding Packet ===");
        // Decode in reverse order
        for (int i = protocols.size()-1; i >= 0; i--) begin
            $display("\n--- Layer %0d Decode ---", i+1);
            protocols[i].decode_packet(data);
        end
    end
    
    function int calculate_total_overhead();
        int total = 0;
        foreach (protocols[i]) begin
            total += protocols[i].get_overhead();
        end
        return total;
    endfunction
endclass

// Test module for network protocols
module network_demo;
    NetworkStack stack;
    bit [7:0] packet_data[];
    
    initial begin
        stack = new();
        packet_data = new[100];  // 100-byte data packet
        
        // Initialize packet with dummy data
        foreach (packet_data[i]) begin
            packet_data[i] = i % 256;
        end
        
        // Process packet through protocol stack
        stack.process_packet(packet_data);
        
        $display("\nTotal Protocol Overhead: %0d bytes", 
                 stack.calculate_total_overhead());
    end
endmodule
```

### Example 3: Employee Management System

```systemverilog
// Payroll system demonstrating polymorphism
class PayrollSystem;
    Employee employees[];
    
    function new();
        Manager mgr;
        Developer dev1, dev2;
        
        // Create different types of employees
        mgr = new("Alice Johnson", 1001, 75000.0, 0.20, 8);
        dev1 = new("Bob Smith", 1002, 65000.0, "SystemVerilog");
        dev2 = new("Carol Davis", 1003, 68000.0, "Python");
        
        // Add some completed projects for developers
        dev1.complete_project();
        dev1.complete_project();
        dev2.complete_project();
        dev2.complete_project();
        dev2.complete_project();
        
        // Store in polymorphic array
        employees = new[3];
        employees[0] = mgr;
        employees[1] = dev1;
        employees[2] = dev2;
    endfunction
    
    function void process_payroll();
        real total_payroll = 0.0;
        
        $display("=== Monthly Payroll Processing ===\n");
        
        foreach (employees[i]) begin
            real pay = employees[i].calculate_pay();  // Polymorphic call
            total_payroll += pay;
            
            employees[i].display_info();  // Polymorphic call
            $display("Monthly Pay: $%0.2f\n", pay);
        end
        
        $display("Total Monthly Payroll: $%0.2f", total_payroll);
    endfunction
    
    function void display_employee_details();
        $display("=== Employee Details ===\n");
        
        foreach (employees[i]) begin
            employees[i].display_info();
            
            // Type checking and casting for specific methods
            if ($cast(mgr_handle, employees[i])) begin
                Manager mgr_handle;
                mgr_handle.conduct_meeting();
            end else if ($cast(dev_handle, employees[i])) begin
                Developer dev_handle;
                dev_handle.complete_project();
            end
            $display("");
        end
    endfunction
endclass

// Test module for payroll system
module payroll_demo;
    PayrollSystem payroll;
    
    initial begin
        payroll = new();
        payroll.process_payroll();
        $display("\n" + {50{"="}});
        payroll.display_employee_details();
    end
endmodule
```

## Best Practices and Design Patterns

### Liskov Substitution Principle
Objects of derived classes should be substitutable for objects of the base class without altering program correctness.

```systemverilog
// Good: Circle can substitute Shape
Shape my_shape = new Circle(5.0);
real area = my_shape.calculate_area();  // Works correctly
```

### Interface Segregation
Keep interfaces focused and cohesive.

```systemverilog
// Instead of one large interface, use specific interfaces
class Drawable;
    pure virtual function void draw();
endclass

class Calculable;
    pure virtual function real calculate_area();
endclass

class Circle extends Drawable, Calculable;  // Multiple inheritance
    // Implementation
endclass
```

### Factory Pattern Example

```systemverilog
class ShapeFactory;
    static function Shape create_shape(string shape_type, real param1 = 1.0, real param2 = 1.0);
        case (shape_type.tolower())
            "circle": return new Circle(param1);
            "rectangle": return new Rectangle(param1, param2);
            "triangle": return new Triangle(param1, param2);
            default: begin
                $display("Unknown shape type: %s", shape_type);
                return null;
            end
        endcase
    endfunction
endclass

// Usage
Shape my_shape = ShapeFactory::create_shape("circle", 3.0);
```

## Common Pitfalls and Debugging Tips

### Forgetting virtual keyword
```systemverilog
// Wrong: Method won't be overridden properly
function void my_method();  // Not virtual

// Correct: Use virtual for overrideable methods
virtual function void my_method();
```

### Incorrect super usage
```systemverilog
// Wrong: Calling super incorrectly
function new();
    super();  // Syntax error

// Correct: Proper super call
function new();
    super.new();  // Correct syntax
```

### Handle assignment vs object copying
```systemverilog
// This copies the handle, not the object
Shape shape1 = new Circle(5.0);
Shape shape2 = shape1;  // Both handles point to same object
```

## Summary

This chapter covered the essential object-oriented programming concepts in SystemVerilog:

- **Inheritance**: Creating new classes based on existing ones using `extends`
- **Method Overriding**: Providing specific implementations in derived classes
- **super keyword**: Accessing parent class methods and properties
- **Virtual Methods**: Enabling runtime method resolution for polymorphism
- **Abstract Classes**: Creating base classes with pure virtual methods
- **Polymorphism**: Treating objects of different types uniformly through common interfaces

These concepts enable you to write more modular, maintainable, and extensible SystemVerilog code by leveraging the power of object-oriented design principles. Understanding inheritance and polymorphism is crucial for building complex verification environments and reusable code libraries.

# SystemVerilog OOP - Simple Examples by Topic

## Class Inheritance (extends)

### **Example 1:** Basic Pet Hierarchy
**Description:** Simple Animal base class with Dog and Cat derived classes. Shows basic inheritance syntax with name and age properties.

In [3]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_1__basic_pet_hierarchy/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// pet_hierarchy_design.sv
// Basic Pet Hierarchy Example - Chapter 10 Example 1

// Base Animal class
class Animal;
  string name;
  int age;
  
  // Constructor
  function new(string pet_name = "Unknown", int pet_age = 0);
    this.name = pet_name;
    this.age = pet_age;
  endfunction
  
  // Virtual method for making sound (to be overridden)
  virtual function string make_sound();
    return "Some generic animal sound";
  endfunction
  
  // Method to display pet information
  function void display_info();
    $display("Pet Name: %s, Age: %d years", name, age);
  endfunction
  
  // Method to get pet description
  virtual function string get_description();
    return $sformatf("%s is %d years old", name, age);
  endfunction
endclass

// Dog class inheriting from Animal
class Dog extends Animal;
  string breed;
  
  // Constructor
  function new(string pet_name = "Buddy", int pet_age = 0, 
               string dog_breed = "Mixed");
    super.new(pet_name, pet_age);  // Call parent constructor
    this.breed = dog_breed;
  endfunction
  
  // Override make_sound method
  function string make_sound();
    return "Woof! Woof!";
  endfunction
  
  // Override get_description method
  function string get_description();
    return $sformatf("%s is a %d-year-old %s", name, age, breed);
  endfunction
  
  // Dog-specific method
  function void wag_tail();
    $display("%s is wagging tail happily!", name);
  endfunction
endclass

// Cat class inheriting from Animal
class Cat extends Animal;
  bit is_indoor;
  
  // Constructor  
  function new(string pet_name = "Whiskers", int pet_age = 0, 
               bit indoor = 1);
    super.new(pet_name, pet_age);  // Call parent constructor
    this.is_indoor = indoor;
  endfunction
  
  // Override make_sound method
  function string make_sound();
    return "Meow! Meow!";
  endfunction
  
  // Override get_description method
  function string get_description();
    string location = is_indoor ? "indoor" : "outdoor";
    return $sformatf("%s is a %d-year-old %s cat", name, age, location);
  endfunction
  
  // Cat-specific method
  function void purr();
    $display("%s is purring contentedly!", name);
  endfunction
endclass

// Top-level module for design
module pet_hierarchy_design;
  // This module serves as a container for the class definitions
  // The actual testing will be done in the testbench
endmodule
```

```systemverilog
// pet_hierarchy_design_testbench.sv
// Testbench for Basic Pet Hierarchy Example

module pet_hierarchy_testbench;
  
  // Instantiate the design module
  pet_hierarchy_design DESIGN_INSTANCE();
  
  // Test variables
  Animal base_animal;
  Dog my_dog;
  Cat my_cat;
  Animal animal_ref;  // Reference for polymorphism demonstration
  
  initial begin
    // Setup waveform dumping
    $dumpfile("pet_hierarchy_testbench.vcd");
    $dumpvars(0, pet_hierarchy_testbench);
    
    $display("=== Pet Hierarchy Inheritance Example ===");
    $display();
    
    // Test 1: Create base Animal
    $display("1. Testing Base Animal Class:");
    base_animal = new("Generic Pet", 5);
    base_animal.display_info();
    $display("Sound: %s", base_animal.make_sound());
    $display("Description: %s", base_animal.get_description());
    $display();
    
    // Test 2: Create Dog instance
    $display("2. Testing Dog Class (inherits from Animal):");
    my_dog = new("Buddy", 3, "Golden Retriever");
    my_dog.display_info();
    $display("Sound: %s", my_dog.make_sound());
    $display("Description: %s", my_dog.get_description());
    my_dog.wag_tail();
    $display();
    
    // Test 3: Create Cat instance
    $display("3. Testing Cat Class (inherits from Animal):");
    my_cat = new("Whiskers", 2, 1);  // Indoor cat
    my_cat.display_info();
    $display("Sound: %s", my_cat.make_sound());
    $display("Description: %s", my_cat.get_description());
    my_cat.purr();
    $display();
    
    // Test 4: Polymorphism demonstration
    $display("4. Testing Polymorphism:");
    $display("Using Animal reference to point to Dog:");
    animal_ref = my_dog;
    $display("Sound via Animal reference: %s", animal_ref.make_sound());
    $display("Description via Animal reference: %s", 
             animal_ref.get_description());
    
    $display("Using Animal reference to point to Cat:");
    animal_ref = my_cat;
    $display("Sound via Animal reference: %s", animal_ref.make_sound());
    $display("Description via Animal reference: %s", 
             animal_ref.get_description());
    $display();
    
    // Test 5: Create different pets with default constructors
    $display("5. Testing Default Constructors:");
    begin
      Dog default_dog;
      Cat default_cat;
      
      default_dog = new();
      default_cat = new();
      
      $display("Default Dog: %s", default_dog.get_description());
      $display("Default Cat: %s", default_cat.get_description());
    end
    
    $display();
    $display("=== All tests completed successfully! ===");
    
    // Small delay before finishing
    #10;
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Pet Hierarchy Inheritance Example ===

1. Testing Base Animal Class:
Pet Name: Generic Pet, Age:           5 years
Sound: Some generic animal sound
Description: Generic Pet is           5 years old

2. Testing Dog Class (inherits from Animal):
Pet Name: Buddy, Age:           3 years
Sound: Woof! Woof!
Description: Buddy is a           3-year-old Golden Retriever
Buddy is wagging tail happily!

3. Testing Cat Class (inherits from Animal):
Pet Name: Whiskers, Age:           2 years
Sound: Meow! Meow!
Description: Whiskers is a           2-year-old indoor cat
Whiskers is purring contentedly!

4. Testing Polymorphism:
Using Animal reference to point to Dog:
Sound via Animal reference: Woof! Woof!
Description via Animal reference: Buddy is a           3-year-old Golden
Retriever
Using Animal reference to point to Cat:
Sound via Animal reference: Meow! Meow!
Description via Animal reference: Whiskers is a           2-year-old indoor cat

5. Testing Default Co

0

### 2. Electronic Device Family
**Description:** Device base class with Phone and Laptop derived classes. Demonstrates inheriting common properties like brand, model, and power_on() method.

In [7]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_2__electronic_device_family/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// electronic_device_family.sv
// Electronic Device Family with inheritance-like behavior

// Base device interface defining common properties and methods
interface device_interface;
  string brand;
  string model;
  bit power_state;
  
  // Common power on method
  task power_on();
    power_state = 1'b1;
    $display("Device %s %s powered ON", brand, model);
  endtask
  
  // Common power off method  
  task power_off();
    power_state = 1'b0;
    $display("Device %s %s powered OFF", brand, model);
  endtask
  
  // Pure virtual method to be implemented by derived classes
  task show_device_info();
    $display("Generic device: %s %s", brand, model);
  endtask
endinterface

// Phone class inheriting from device interface
module phone_device(device_interface dev_if);
  string phone_number = "+1-555-0123";
  bit has_camera = 1'b1;
  
  initial begin
    dev_if.brand = "Apple";
    dev_if.model = "iPhone 15";
    #1; // Small delay to ensure assignment
  end
  
  // Override device info method
  task show_phone_info();
    $display("Phone: %s %s", dev_if.brand, dev_if.model);
    $display("  Phone Number: %s", phone_number);
    $display("  Has Camera: %s", has_camera ? "Yes" : "No");
  endtask
  
  // Phone-specific method
  task make_call(string number);
    if (dev_if.power_state) begin
      $display("Calling %s from %s...", number, phone_number);
    end else begin
      $display("Cannot make call - phone is powered off");
    end
  endtask
endmodule

// Laptop class inheriting from device interface
module laptop_device(device_interface dev_if);
  string operating_system = "Ubuntu Linux";
  int ram_gb = 16;
  
  initial begin
    dev_if.brand = "Dell";
    dev_if.model = "XPS 13";
    #1; // Small delay to ensure assignment
  end
  
  // Override device info method
  task show_laptop_info();
    $display("Laptop: %s %s", dev_if.brand, dev_if.model);
    $display("  Operating System: %s", operating_system);
    $display("  RAM: %0d GB", ram_gb);
  endtask
  
  // Laptop-specific method
  task run_application(string app_name);
    if (dev_if.power_state) begin
      $display("Running %s on %s %s", app_name, dev_if.brand, 
               dev_if.model);
    end else begin
      $display("Cannot run application - laptop is powered off");
    end
  endtask
endmodule

// Top-level design module
module electronic_device_family();
  // Create device interfaces
  device_interface phone_if();
  device_interface laptop_if();
  
  // Instantiate device modules
  phone_device phone_inst(phone_if);
  laptop_device laptop_inst(laptop_if);
  
  initial begin
    $display("=== Electronic Device Family Demo ===");
    $display();
    
    // Wait for initialization
    #2;
    
    // Show initial device information
    phone_inst.show_phone_info();
    $display();
    laptop_inst.show_laptop_info();
    $display();
    
    // Test common power methods
    $display("--- Testing Power Control ---");
    phone_if.power_on();
    laptop_if.power_on();
    $display();
    
    // Test device-specific methods
    $display("--- Testing Device-Specific Methods ---");
    phone_inst.make_call("+1-555-9876");
    laptop_inst.run_application("Firefox Browser");
    $display();
    
    // Test power off
    $display("--- Testing Power Off ---");
    phone_if.power_off();
    laptop_if.power_off();
    $display();
    
    // Try to use devices when powered off
    $display("--- Testing Powered Off State ---");
    phone_inst.make_call("+1-555-5555");
    laptop_inst.run_application("Text Editor");
  end
endmodule
```

```systemverilog
// electronic_device_family_testbench.sv
// Testbench for Electronic Device Family demonstration

module device_family_testbench;
  // Instantiate the design under test
  electronic_device_family DEVICE_FAMILY_INSTANCE();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("device_family_testbench.vcd");
    $dumpvars(0, device_family_testbench);
    
    // Wait for design to complete
    #100;
    
    $display();
    $display("=== Additional Testbench Verification ===");
    $display();
    
    // Test power state verification
    $display("--- Power State Verification ---");
    $display("Phone power state: %s", 
             DEVICE_FAMILY_INSTANCE.phone_if.power_state ? "ON" : "OFF");
    $display("Laptop power state: %s",
             DEVICE_FAMILY_INSTANCE.laptop_if.power_state ? "ON" : "OFF");
    $display();
    
    // Test brand and model access
    $display("--- Device Properties Access ---");
    $display("Phone brand: %s, model: %s", 
             DEVICE_FAMILY_INSTANCE.phone_if.brand,
             DEVICE_FAMILY_INSTANCE.phone_if.model);
    $display("Laptop brand: %s, model: %s",
             DEVICE_FAMILY_INSTANCE.laptop_if.brand,
             DEVICE_FAMILY_INSTANCE.laptop_if.model);
    $display();
    
    // Test polymorphic behavior
    $display("--- Polymorphic Method Testing ---");
    DEVICE_FAMILY_INSTANCE.phone_if.show_device_info();
    DEVICE_FAMILY_INSTANCE.laptop_if.show_device_info();
    $display();
    
    $display("Hello from device family testbench!");
    $display();
    
    // End simulation
    $finish;
  end
endmodule
```

Verilator Simulation Output:
=== Electronic Device Family Demo ===

Phone: Apple iPhone 15
  Phone Number: +1-555-0123
  Has Camera: Yes

Laptop: Dell XPS 13
  Operating System: Ubuntu Linux
  RAM: 16 GB

--- Testing Power Control ---
Device Apple iPhone 15 powered ON
Device Dell XPS 13 powered ON

--- Testing Device-Specific Methods ---
Calling +1-555-9876 from +1-555-0123...
Running Firefox Browser on Dell XPS 13

--- Testing Power Off ---
Device Apple iPhone 15 powered OFF
Device Dell XPS 13 powered OFF

--- Testing Powered Off State ---
Cannot make call - phone is powered off
Cannot run application - laptop is powered off

=== Additional Testbench Verification ===

--- Power State Verification ---
Phone power state: OFF
Laptop power state: OFF

--- Device Properties Access ---
Phone brand: Apple, model: iPhone 15
Laptop brand: Dell, model: XPS 13

--- Polymorphic Method Testing ---
Generic device: Apple iPhone 15
Generic device: Dell XPS 13

Hello from device family testbench!

Pro

0

### 3. Simple Food Chain
**Description:** Food base class with Fruit and Vegetable derived classes. Shows inheritance of basic properties like name, color, and nutritional_value.

In [12]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_3__simple_food_chain/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// food_chain_classes.sv
package food_chain_pkg;

  // Base Food class
  virtual class Food;
    string name;
    string color;
    int nutritional_value;
    
    // Constructor
    function new(string n, string c, int nv);
      name = n;
      color = c;
      nutritional_value = nv;
    endfunction
    
    // Virtual method to display info
    virtual function void display_info();
      $display("Food: %s, Color: %s, Nutritional Value: %0d", 
               name, color, nutritional_value);
    endfunction
    
    // Virtual method to get food type
    virtual function string get_food_type();
      return "Generic Food";
    endfunction
  endclass

  // Fruit class derived from Food
  class Fruit extends Food;
    bit is_sweet;
    string season;
    
    // Constructor
    function new(string n, string c, int nv, bit sweet, string s);
      super.new(n, c, nv);
      is_sweet = sweet;
      season = s;
    endfunction
    
    // Override display_info method
    virtual function void display_info();
      $display("Fruit: %s, Color: %s, Nutritional Value: %0d, Sweet: %s, Season: %s",
               name, color, nutritional_value, 
               is_sweet ? "Yes" : "No", season);
    endfunction
    
    // Override get_food_type method
    virtual function string get_food_type();
      return "Fruit";
    endfunction
    
    // Fruit-specific method
    function void ripen();
      $display("%s is ripening...", name);
    endfunction
  endclass

  // Vegetable class derived from Food
  class Vegetable extends Food;
    string part_of_plant;
    bit needs_cooking;
    
    // Constructor
    function new(string n, string c, int nv, string part, bit cooking);
      super.new(n, c, nv);
      part_of_plant = part;
      needs_cooking = cooking;
    endfunction
    
    // Override display_info method
    virtual function void display_info();
      $display("Vegetable: %s, Color: %s, Nutritional Value: %0d, Part: %s, Needs Cooking: %s",
               name, color, nutritional_value, part_of_plant,
               needs_cooking ? "Yes" : "No");
    endfunction
    
    // Override get_food_type method
    virtual function string get_food_type();
      return "Vegetable";
    endfunction
    
    // Vegetable-specific method
    function void harvest();
      $display("%s is being harvested from the %s", name, part_of_plant);
    endfunction
  endclass

endpackage

module food_chain_classes();

  initial begin
    $display("Food Chain Classes - Design Module");
    $display("=====================================");
    $display("Classes defined in food_chain_pkg package");
    $display();
  end

endmodule
```

```systemverilog
// food_chain_classes_testbench.sv
module food_chain_testbench;

  // Import the food chain package
  import food_chain_pkg::*;

  // Instantiate design under test
  food_chain_classes FOOD_CHAIN_INSTANCE();

  // Declare all class handles and variables at module level
  Fruit apple, banana, orange;
  Vegetable carrot, spinach, potato;
  Food food_handle;
  food_chain_pkg::Fruit scoped_apple;
  int total_nutrition;

  // Task to create and initialize all food objects
  task create_food_objects();
    apple = new("Apple", "Red", 95, 1, "Fall");
    banana = new("Banana", "Yellow", 105, 1, "Year-round");
    orange = new("Orange", "Orange", 85, 1, "Winter");
    carrot = new("Carrot", "Orange", 25, "Root", 0);
    spinach = new("Spinach", "Green", 15, "Leaf", 0);
    potato = new("Potato", "Brown", 160, "Root", 1);
    scoped_apple = new("Scoped Apple", "Green", 80, 1, "Summer");
  endtask

  // Task to display fruit information
  task display_fruits();
    $display("Fruits Information:");
    $display("------------------");
    apple.display_info();
    banana.display_info();
    orange.display_info();
    $display();
  endtask

  // Task to display vegetable information
  task display_vegetables();
    $display("Vegetables Information:");
    $display("----------------------");
    carrot.display_info();
    spinach.display_info();
    potato.display_info();
    $display();
  endtask

  // Task to demonstrate fruit operations
  task demonstrate_fruit_operations();
    $display("Fruit Operations:");
    $display("----------------");
    apple.ripen();
    banana.ripen();
    orange.ripen();
    $display();
  endtask

  // Task to demonstrate vegetable operations
  task demonstrate_vegetable_operations();
    $display("Vegetable Operations:");
    $display("--------------------");
    carrot.harvest();
    spinach.harvest();
    potato.harvest();
    $display();
  endtask

  // Task to calculate and display nutritional information
  task display_nutrition_summary();
    $display("Nutritional Summary:");
    $display("-------------------");
    
    total_nutrition = 0;
    total_nutrition += apple.nutritional_value;
    $display("Apple: %0d calories", apple.nutritional_value);
    
    total_nutrition += banana.nutritional_value;
    $display("Banana: %0d calories", banana.nutritional_value);
    
    total_nutrition += orange.nutritional_value;
    $display("Orange: %0d calories", orange.nutritional_value);
    
    total_nutrition += carrot.nutritional_value;
    $display("Carrot: %0d calories", carrot.nutritional_value);
    
    total_nutrition += spinach.nutritional_value;
    $display("Spinach: %0d calories", spinach.nutritional_value);
    
    total_nutrition += potato.nutritional_value;
    $display("Potato: %0d calories", potato.nutritional_value);
    
    $display("Total Calories: %0d", total_nutrition);
    $display();
  endtask

  // Task to test polymorphism
  task test_polymorphism();
    $display("Polymorphism Test:");
    $display("-----------------");
    
    food_handle = apple;
    $display("Base handle pointing to apple:");
    $display("Type: %s", food_handle.get_food_type());
    food_handle.display_info();
    $display();
    
    food_handle = carrot;
    $display("Base handle pointing to carrot:");
    $display("Type: %s", food_handle.get_food_type());
    food_handle.display_info();
    $display();
  endtask

  // Task to test package scope
  task test_package_scope();
    $display("Package Scope Test:");
    $display("------------------");
    scoped_apple.display_info();
    $display();
  endtask

  // Main test sequence
  initial begin
    // Dump waves
    $dumpfile("food_chain_testbench.vcd");
    $dumpvars(0, food_chain_testbench);
    
    $display("Food Chain Inheritance Example");
    $display("==============================");
    $display();
    
    // Create all food objects
    create_food_objects();
    
    // Display information
    display_fruits();
    display_vegetables();
    
    // Demonstrate operations
    demonstrate_fruit_operations();
    demonstrate_vegetable_operations();
    
    // Show nutritional summary
    display_nutrition_summary();
    
    // Test polymorphism
    test_polymorphism();
    
    // Test package scope
    test_package_scope();
    
    $display("Food Chain Example Complete!");
    $display("============================");
    
    #1;  // Wait for a time unit
  end

endmodule
```

Verilator Simulation Output:
Food Chain Classes - Design Module
Classes defined in food_chain_pkg package

Food Chain Inheritance Example

Fruits Information:
------------------
Fruit: Apple, Color: Red, Nutritional Value: 95, Sweet: Yes, Season: Fall
Fruit: Banana, Color: Yellow, Nutritional Value: 105, Sweet: Yes, Season: Year-
round
Fruit: Orange, Color: Orange, Nutritional Value: 85, Sweet: Yes, Season: Winter

Vegetables Information:
----------------------
Vegetable: Carrot, Color: Orange, Nutritional Value: 25, Part: Root, Needs
Cooking:  No
Vegetable: Spinach, Color: Green, Nutritional Value: 15, Part: Leaf, Needs
Cooking:  No
Vegetable: Potato, Color: Brown, Nutritional Value: 160, Part: Root, Needs
Cooking: Yes

Fruit Operations:
----------------
Apple is ripening...
Banana is ripening...
Orange is ripening...

Vegetable Operations:
--------------------
Carrot is being harvested from the Root
Spinach is being harvested from the Leaf
Potato is being harvested from the Root

Nut

0

## Method Overriding

### 4. Sound Maker Animals
**Description:** Animal base class with make_sound() method. Dog, Cat, and Cow classes override to make "Woof!", "Meow!", and "Moo!" respectively.

In [17]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_4__sound_maker_animals/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// animal_sound_maker.sv
package animal_sound_pkg;

  // Animal type constants
  parameter int ANIMAL_DOG = 0;
  parameter int ANIMAL_CAT = 1;
  parameter int ANIMAL_COW = 2;
  parameter int ANIMAL_GENERIC = 3;

  // Function to make sound based on animal type
  function automatic void make_sound(int animal_type, string name);
    case (animal_type)
      ANIMAL_DOG: $display("%s says: Woof!", name);
      ANIMAL_CAT: $display("%s says: Meow!", name);
      ANIMAL_COW: $display("%s says: Moo!", name);
      ANIMAL_GENERIC: $display("%s makes a generic animal sound", name);
      default: $display("%s makes an unknown sound", name);
    endcase
  endfunction

  // Function to introduce animal
  function automatic void introduce(int animal_type, string name);
    string type_str;
    case (animal_type)
      ANIMAL_DOG: type_str = "Dog";
      ANIMAL_CAT: type_str = "Cat";
      ANIMAL_COW: type_str = "Cow";
      ANIMAL_GENERIC: type_str = "Animal";
      default: type_str = "Unknown";
    endcase
    $display("I am %s, a %s", name, type_str);
  endfunction

  // Function to get animal type string
  function automatic string get_type_string(int animal_type);
    case (animal_type)
      ANIMAL_DOG: return "Dog";
      ANIMAL_CAT: return "Cat";
      ANIMAL_COW: return "Cow";
      ANIMAL_GENERIC: return "Animal";
      default: return "Unknown";
    endcase
  endfunction

endpackage

module animal_sound_maker();
  import animal_sound_pkg::*;

  initial begin
    $display("=== Animal Sound Maker Demo ===");
    $display();
  end

endmodule
```

```systemverilog
// animal_sound_maker_testbench.sv
module animal_sound_testbench;
  import animal_sound_pkg::*;

  // Instantiate design under test
  animal_sound_maker ANIMAL_SOUND_INSTANCE();

  // Declare all variables at module level for Verilator compatibility
  int buddy_type;
  string buddy_name;
  int whiskers_type;
  string whiskers_name;
  int bessie_type;
  string bessie_name;
  
  // Arrays for farm animals
  int farm_animal_types[3];
  string farm_animal_names[3];
  int i;

  initial begin
    // Dump waves for debugging
    $dumpfile("animal_sound_testbench.vcd");
    $dumpvars(0, animal_sound_testbench);
    
    $display("=== Creating Animal Objects ===");
    buddy_type = ANIMAL_DOG;
    buddy_name = "Buddy";
    
    whiskers_type = ANIMAL_CAT;
    whiskers_name = "Whiskers";
    
    bessie_type = ANIMAL_COW;
    bessie_name = "Bessie";
    
    $display();
    $display("=== Individual Animal Sounds ===");
    
    // Test individual animals
    introduce(buddy_type, buddy_name);
    make_sound(buddy_type, buddy_name);
    $display();
    
    introduce(whiskers_type, whiskers_name);
    make_sound(whiskers_type, whiskers_name);
    $display();
    
    introduce(bessie_type, bessie_name);
    make_sound(bessie_type, bessie_name);
    $display();
    
    // Demonstrate polymorphism with arrays
    $display("=== Polymorphism Demo ===");
    farm_animal_types[0] = buddy_type;
    farm_animal_names[0] = buddy_name;
    farm_animal_types[1] = whiskers_type;
    farm_animal_names[1] = whiskers_name;
    farm_animal_types[2] = bessie_type;
    farm_animal_names[2] = bessie_name;
    
    $display("Farm animals making sounds:");
    for (i = 0; i < 3; i = i + 1) begin
      make_sound(farm_animal_types[i], farm_animal_names[i]);
    end
    
    $display();
    $display("=== Sound Maker Animals Test Complete ===");
    
    #10; // Wait a bit before finishing
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Animal Sound Maker Demo ===

=== Creating Animal Objects ===

=== Individual Animal Sounds ===
I am Buddy, a Dog
Buddy says: Woof!

I am Whiskers, a Cat
Whiskers says: Meow!

I am Bessie, a Cow
Bessie says: Moo!

=== Polymorphism Demo ===
Farm animals making sounds:
Buddy says: Woof!
Whiskers says: Meow!
Bessie says: Moo!

=== Sound Maker Animals Test Complete ===
Process finished with return code: 0
Removing Chapter_10_examples/example_4__sound_maker_animals/obj_dir directory...
Chapter_10_examples/example_4__sound_maker_animals/obj_dir removed successfully.


0

### 5. Transportation Methods
**Description:** Transport base class with move() method. Car, Bike, and Plane classes override to show "Driving...", "Pedaling...", and "Flying...".

In [20]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_5__transportation_methods/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// transportation_methods.sv
// Transportation base class and derived classes example

// Base Transportation class
virtual class Transportation;
  string name;
  
  function new(string transport_name);
    this.name = transport_name;
  endfunction
  
  // Virtual method to be overridden by derived classes
  virtual function void move();
    $display("Generic transportation moving...");
  endfunction
  
  virtual function void show_info();
    $display("Transportation: %s", name);
  endfunction
endclass

// Car class - derived from Transportation
class Car extends Transportation;
  function new(string car_name);
    super.new(car_name);
  endfunction
  
  // Override move method
  virtual function void move();
    $display("Driving...");
  endfunction
endclass

// Bike class - derived from Transportation
class Bike extends Transportation;
  function new(string bike_name);
    super.new(bike_name);
  endfunction
  
  // Override move method
  virtual function void move();
    $display("Pedaling...");
  endfunction
endclass

// Plane class - derived from Transportation
class Plane extends Transportation;
  function new(string plane_name);
    super.new(plane_name);
  endfunction
  
  // Override move method
  virtual function void move();
    $display("Flying...");
  endfunction
endclass

// Transportation manager module
module transportation_manager();
  Car my_car;
  Bike my_bike;
  Plane my_plane;
  
  initial begin
    $display("=== Transportation Methods Example ===");
    $display();
    
    // Create instances of different transportation types
    my_car = new("Toyota Camry");
    my_bike = new("Mountain Bike");
    my_plane = new("Boeing 737");
    
    // Show info and demonstrate move methods
    my_car.show_info();
    my_car.move();
    $display();
    
    my_bike.show_info();
    my_bike.move();
    $display();
    
    my_plane.show_info();
    my_plane.move();
    $display();
    
    $display("=== Polymorphism Example ===");
    demonstrate_polymorphism();
  end
  
  // Function to demonstrate polymorphism
  function void demonstrate_polymorphism();
    Transportation transports[];
    Car poly_car;
    Bike poly_bike;
    Plane poly_plane;
    
    // Create array of different transportation types
    transports = new[3];
    
    // Create individual instances first
    poly_car = new("Honda Civic");
    poly_bike = new("Road Bike");
    poly_plane = new("Airbus A320");
    
    // Assign to base class array
    transports[0] = poly_car;
    transports[1] = poly_bike;
    transports[2] = poly_plane;
    
    // Call move() on each - polymorphic behavior
    foreach(transports[i]) begin
      transports[i].show_info();
      transports[i].move();
      $display();
    end
  endfunction
endmodule
```

```systemverilog
// transportation_methods_testbench.sv
// Testbench for transportation methods example

module transportation_testbench;
  // Instantiate the transportation manager
  transportation_manager TRANSPORT_INSTANCE();
  
  initial begin
    // Dump waves for Verilator
    $dumpfile("transportation_testbench.vcd");
    $dumpvars(0, transportation_testbench);
    
    #1; // Wait for design to execute
    
    $display("=== Additional Transportation Tests ===");
    test_individual_transports();
    
    $display("=== Test Complete ===");
    $display();
  end
  
  // Additional test function
  function void test_individual_transports();
    Car test_car;
    Bike test_bike;
    Plane test_plane;
    
    $display("Testing individual transportation objects:");
    $display();
    
    // Test Car
    test_car = new("Test Car");
    $display("Creating and testing %s:", test_car.name);
    test_car.move();
    $display();
    
    // Test Bike
    test_bike = new("Test Bike");
    $display("Creating and testing %s:", test_bike.name);
    test_bike.move();
    $display();
    
    // Test Plane
    test_plane = new("Test Plane");
    $display("Creating and testing %s:", test_plane.name);
    test_plane.move();
    $display();
  endfunction
endmodule
```

Verilator Simulation Output:
=== Transportation Methods Example ===

Transportation: Toyota Camry
Driving...

Transportation: Mountain Bike
Pedaling...

Transportation: Boeing 737
Flying...

=== Polymorphism Example ===
Transportation: Honda Civic
Driving...

Transportation: Road Bike
Pedaling...

Transportation: Airbus A320
Flying...

=== Additional Transportation Tests ===
Testing individual transportation objects:

Creating and testing Test Car:
Driving...

Creating and testing Test Bike:
Pedaling...

Creating and testing Test Plane:
Flying...

=== Test Complete ===
Process finished with return code: 0
Removing Chapter_10_examples/example_5__transportation_methods/obj_dir directory...
Chapter_10_examples/example_5__transportation_methods/obj_dir removed successfully.


0

### 6. Simple Math Operations
**Description:** Calculator base class with calculate() method. Add, Subtract, and Multiply classes override to perform specific operations.

In [22]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_6__simple_math_operations/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_math_calculator.sv
module simple_math_calculator_module();

  // Base Calculator class with virtual calculate method
  virtual class Calculator;
    protected int operand_a;
    protected int operand_b;
    
    function new(int a, int b);
      operand_a = a;
      operand_b = b;
    endfunction
    
    // Virtual method to be overridden by derived classes
    pure virtual function int calculate();
    
    function void display_operands();
      $display("Operand A: %0d, Operand B: %0d", operand_a, operand_b);
    endfunction
  endclass

  // Add class - performs addition
  class Add extends Calculator;
    function new(int a, int b);
      super.new(a, b);
    endfunction
    
    virtual function int calculate();
      return operand_a + operand_b;
    endfunction
  endclass

  // Subtract class - performs subtraction
  class Subtract extends Calculator;
    function new(int a, int b);
      super.new(a, b);
    endfunction
    
    virtual function int calculate();
      return operand_a - operand_b;
    endfunction
  endclass

  // Multiply class - performs multiplication
  class Multiply extends Calculator;
    function new(int a, int b);
      super.new(a, b);
    endfunction
    
    virtual function int calculate();
      return operand_a * operand_b;
    endfunction
  endclass

  initial begin
    Add add_calc;
    Subtract sub_calc;
    Multiply mul_calc;
    Calculator calc_handle;
    int result;
    
    $display();
    $display("=== Simple Math Operations Example ===");
    $display();
    
    // Test Addition
    add_calc = new(15, 7);
    calc_handle = add_calc;
    calc_handle.display_operands();
    result = calc_handle.calculate();
    $display("Addition Result: %0d", result);
    $display();
    
    // Test Subtraction
    sub_calc = new(20, 8);
    calc_handle = sub_calc;
    calc_handle.display_operands();
    result = calc_handle.calculate();
    $display("Subtraction Result: %0d", result);
    $display();
    
    // Test Multiplication
    mul_calc = new(6, 9);
    calc_handle = mul_calc;
    calc_handle.display_operands();
    result = calc_handle.calculate();
    $display("Multiplication Result: %0d", result);
    $display();
    
    $display("=== Math Operations Complete ===");
  end

endmodule
```

```systemverilog
// simple_math_calculator_testbench.sv
module test_bench_module;

  // Instantiate design under test
  simple_math_calculator_module CALCULATOR_INSTANCE();

  initial begin
    // Dump waves for debugging
    $dumpfile("test_bench_module.vcd");
    $dumpvars(0, test_bench_module);
    
    $display();
    $display("=== Math Calculator Testbench Started ===");
    $display();
    
    // Wait for design to complete
    #10;
    
    $display();
    $display("=== All Math Operations Verified ===");
    $display();
    
    // End simulation
    $finish;
  end

endmodule
```

Verilator Simulation Output:

=== Math Calculator Testbench Started ===


=== Simple Math Operations Example ===

Operand A: 15, Operand B: 7
Addition Result: 22

Operand A: 20, Operand B: 8
Subtraction Result: 12

Operand A: 6, Operand B: 9
Multiplication Result: 54

=== Math Operations Complete ===

=== All Math Operations Verified ===

Process finished with return code: 0
Removing Chapter_10_examples/example_6__simple_math_operations/obj_dir directory...
Chapter_10_examples/example_6__simple_math_operations/obj_dir removed successfully.


0

## The super Keyword

### 7. Student Grade System
**Description:** Student base class with name and basic info. GraduateStudent extends it, using super.new() to call parent constructor and super methods for grade calculation.

In [28]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_7__student_grade_system/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// student_grade_system.sv

// Base Student class package
package student_pkg;

class Student;
  protected string name;
  protected int age;
  protected real base_grade;
  
  // Constructor
  function new(string student_name, int student_age, real grade);
    this.name = student_name;
    this.age = student_age;
    this.base_grade = grade;
    $display("Student created: %s, Age: %0d", name, age);
  endfunction
  
  // Virtual method for grade calculation
  virtual function real calculate_final_grade();
    return base_grade;
  endfunction
  
  // Method to display student info
  virtual function void display_info();
    $display("Student: %s, Age: %0d, Final Grade: %.2f", 
             name, age, calculate_final_grade());
  endfunction
  
  // Getter methods
  function string get_name();
    return name;
  endfunction
  
  function int get_age();
    return age;
  endfunction
  
endclass

endpackage : student_pkg

// Graduate Student class package
package graduate_student_pkg;

// Import the base student package
import student_pkg::*;

class GraduateStudent extends Student;
  protected real research_bonus;
  protected real thesis_score;
  
  // Constructor using super.new()
  function new(string student_name, int student_age, real grade, 
               real bonus, real thesis);
    super.new(student_name, student_age, grade);
    this.research_bonus = bonus;
    this.thesis_score = thesis;
    $display("Graduate Student specialized initialization complete");
  endfunction
  
  // Override grade calculation using super method
  virtual function real calculate_final_grade();
    real base_result;
    real final_result;
    
    // Call parent method using super
    base_result = super.calculate_final_grade();
    
    // Add graduate-specific calculations
    final_result = base_result + research_bonus + (thesis_score * 0.1);
    
    // Cap at 100
    if (final_result > 100.0) final_result = 100.0;
    
    return final_result;
  endfunction
  
  // Override display method using super
  virtual function void display_info();
    $display("=== Graduate Student Info ===");
    super.display_info();
    $display("Research Bonus: %.2f, Thesis Score: %.2f", 
             research_bonus, thesis_score);
  endfunction
  
endclass

endpackage : graduate_student_pkg
```

```systemverilog
// student_grade_system_testbench.sv
module student_grade_testbench;
  
  // Import both packages
  import student_pkg::*;
  import graduate_student_pkg::*;
  
  // Test variables
  Student regular_student;
  GraduateStudent grad_student;
  
  initial begin
    // Dump waves
    $dumpfile("student_grade_testbench.vcd");
    $dumpvars(0, student_grade_testbench);
    
    $display("=== Student Grade System Test ===");
    $display();
    
    // Test 1: Create regular student
    $display("Test 1: Creating regular student");
    regular_student = new("Alice Johnson", 20, 85.5);
    regular_student.display_info();
    $display();
    
    // Test 2: Create graduate student using super.new()
    $display("Test 2: Creating graduate student");
    grad_student = new("Bob Smith", 24, 82.0, 5.0, 90.0);
    grad_student.display_info();
    $display();
    
    // Test 3: Compare grade calculations
    $display("Test 3: Grade comparison");
    $display("Regular student '%s' final grade: %.2f", 
             regular_student.get_name(), 
             regular_student.calculate_final_grade());
    $display("Graduate student '%s' final grade: %.2f", 
             grad_student.get_name(), 
             grad_student.calculate_final_grade());
    $display();
    
    // Test 4: Polymorphism test
    $display("Test 4: Polymorphism test");
    begin
      Student student_ref;
      
      // Point to regular student
      student_ref = regular_student;
      $display("Via Student reference (regular):");
      student_ref.display_info();
      $display();
      
      // Point to graduate student
      student_ref = grad_student;
      $display("Via Student reference (graduate):");
      student_ref.display_info();
      $display();
    end
    
    // Test 5: Edge case - high grades
    $display("Test 5: High grade capping test");
    begin
      GraduateStudent high_achiever;
      high_achiever = new("Charlie Brown", 26, 95.0, 8.0, 95.0);
      high_achiever.display_info();
      $display();
    end
    
    $display("=== All tests completed ===");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

Test 1: Creating regular student
Student created: Alice Johnson, Age: 20
Student: Alice Johnson, Age: 20, Final Grade: 85.50

Test 2: Creating graduate student
Student created: Bob Smith, Age: 24
Graduate Student specialized initialization complete
=== Graduate Student Info ===
Student: Bob Smith, Age: 24, Final Grade: 96.00
Research Bonus: 5.00, Thesis Score: 90.00

Test 3: Grade comparison
Regular student 'Alice Johnson' final grade: 85.50
Graduate student 'Bob Smith' final grade: 96.00

Test 4: Polymorphism test
Via Student reference (regular):
Student: Alice Johnson, Age: 20, Final Grade: 85.50

Via Student reference (graduate):
=== Graduate Student Info ===
Student: Bob Smith, Age: 24, Final Grade: 96.00
Research Bonus: 5.00, Thesis Score: 90.00

Test 5: High grade capping test
Student created: Charlie Brown, Age: 26
Graduate Student specialized initialization complete
=== Graduate Student Info ===
Student: Charlie Brown, Age: 26, Final Grade: 100.00


0

### 8. Bank Account Hierarchy
**Description:** Account base class with balance operations. SavingsAccount uses super to call parent methods while adding interest calculation.

### 9. Simple Game Characters
**Description:** Character base class with health and name. Warrior class uses super to initialize base stats then adds specific warrior abilities.

## Virtual Methods

### 10. Simple Drawing Shapes
**Description:** Shape base class with virtual draw() method. Circle, Square, and Triangle override to show different ASCII art representations.

### 11. Basic File Handlers
**Description:** FileHandler base class with virtual process() method. TextFile, ImageFile, and VideoFile classes provide specific processing logic.

### 12. Simple Game Units
**Description:** Unit base class with virtual attack() method. Archer, Knight, and Wizard override to show different attack patterns and damage.

## Abstract Classes (Pure Virtual)

### 13. Basic Database Operations
**Description:** Database base class with pure virtual connect(), query(), and disconnect() methods. MySQL and SQLite classes implement these methods.

### 14. Simple Printer Interface
**Description:** Printer base class with pure virtual print() method. LaserPrinter and InkjetPrinter implement specific printing mechanisms.

### 15. Basic Sensor Interface
**Description:** Sensor base class with pure virtual read_value() method. TemperatureSensor and HumiditySensor implement specific reading logic.

## Polymorphism Examples

### 16. Pet Care System
**Description:** Array of Animal handles containing different pet types. Demonstrates polymorphic method calls for feeding and care routines.

### 17. Simple Media Player
**Description:** MediaFile array with different file types (Audio, Video, Image). Shows polymorphic play() method calls for different media types.

### 18. Basic Shape Calculator
**Description:** Collection of Shape objects calculating total area. Demonstrates polymorphic area calculation for mixed shape types.

### 19. Simple Command Processor
**Description:** Command base class with execute() method. Different command types (Save, Load, Delete) stored in array and executed polymorphically.

### 20. Basic Vehicle Fleet
**Description:** Vehicle array with cars, trucks, and motorcycles. Shows polymorphic fuel calculation and maintenance scheduling.

## Design Patterns

### 21. Simple Factory Pattern
**Description:** AnimalFactory creates different animals based on string input. Demonstrates object creation without exposing instantiation logic.

### 22. Basic Observer Pattern
**Description:** Simple WeatherStation notifies multiple Display objects. Shows basic publisher-subscriber relationship using inheritance.

### 23. Simple Strategy Pattern
**Description:** SortStrategy base class with different sorting algorithms. Context class uses different strategies polymorphically.

## Common Pitfalls Examples

### 24. Virtual Keyword Demo
**Description:** Side-by-side comparison showing difference between virtual and non-virtual methods in inheritance hierarchy.

### 25. Handle vs Object Copying
**Description:** Simple example demonstrating the difference between copying object handles and actual object duplication.

### 26. Proper super Usage
**Description:** Shows correct and incorrect ways to call parent constructors and methods using the super keyword.